In [2]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200


from paths import *
from scpts import manipulation
# from scripts import io

import basedosdados as bd
from basedosdados import Storage, Table, Dataset

import os

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="../../credentials/cli-adm2.json"

import yaml

In [9]:
from google.cloud import bigquery

In [10]:
bigquery.__version__

'1.28.0'

In [3]:
tb = Table('test','br_inep_test_2')

In [7]:
tb.create('test/', if_storage_data_exists='replace',if_table_config_exists='replace',if_table_exists='replace')

Uploading files: 100%|██████████| 1/1 [00:18<00:00, 18.14s/it]


In [11]:
df = pd.read_csv('municipios.csv', error_bad_lines=False, encoding='latin1', sep=';')

In [12]:
df = df.drop(['ano'],1)

In [13]:
df.columns = manipulation.normalize_cols(df.columns)
df.to_csv('test/ano=2006/municipios_solved.csv', index=False)

In [16]:
df = pd.DataFrame([1,2,3,4,5], columns=['test'])

In [17]:
df.to_csv('test_table.csv', index=False)

In [26]:
tb = bd.Table('c_test','a_proposal_test')

In [32]:
!ls

caged.ipynb	      futebol.ipynb	      __pycache__    test_table.csv
censo_2010.ipynb      paths.py		      sandbox.ipynb
create_configs.ipynb  proposal_ambient.ipynb  test_data.csv


In [37]:
tb.create(
    path='test_data.csv',
    if_table_exists='raise',
    if_storage_data_exists='replace',
    if_table_config_exists='raise'
)

Uploading files: 100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


In [38]:
tb.publish()

In [40]:
tb = bd.Table('bens_candidato','br_tse_eleicoes')

In [41]:
# tb.create(
#     partitioned=True,
#     if_table_exists='replace',
#     if_storage_data_exists='pass',
#     if_table_config_exists='replace',
# )

In [42]:
tb.publish()

In [3]:
tb = Table('despesas','br_sp_gov_orcamento')

In [4]:
Dataset('br_sp_gov_orcamento').update()

In [11]:
import pandas as pd
import plotly.express as px

### Orcamento

### Despesas

In [5]:
raw_path = 'data/orcamento/raw/despesa/'
clean_path = 'data/orcamento/clean/despesa/'
files = os.listdir(raw_path)

In [13]:
n=0
for file in files:
    df = pd.read_csv(raw_path+file, encoding='iso-8859-9')
    df.columns = manipulation.normalize_cols(df.columns)

    mask = df['orgao'].notnull()
    df = df[mask]

    drop_cols = [col for col in df.columns.tolist() if 'unnamed' in col]

    df = df.drop(drop_cols, 1)

    numeric_cols = ['dotacao_inicial', 'dotacao_atual', 'empenhado', 'liquidado', 'pago','pago_restos']

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col].str.replace('.','').str.replace(',','.'), errors='coerce')
    
    df['ano'] = int(file[-8:-4])
    
#     print(file, df.columns.tolist())
    df = df.rename(columns={'sub_funcao':'subfuncao'})
    n+=df.shape[0]
    
    df.to_csv(clean_path+file, index=False, encoding='utf-8')

In [70]:
## Cria objeto com table_id e dataset_id
tb = Table('despesas','br_sp_gov_orcamento')

In [71]:
## Faz upload para o bucket e cria arquivos de configuração na pasta bases/dataset_id/table_id
tb.create('data/orcamento/clean/despesa/',if_exists="replace")

In [72]:
tb.publish('replace')


In [14]:
# tb.update()

### Receita

In [2]:
raw_path = 'data/orcamento/raw/receita/'
clean_path = 'data/orcamento/clean/receita/'
files = os.listdir(raw_path)

In [3]:
for file in files:
    df = pd.read_csv(raw_path+file, encoding='iso-8859-9')
    df.columns = manipulation.normalize_cols(df.columns)

    mask = df['orgao'].notnull()
    df = df[mask]

    drop_cols = [col for col in df.columns.tolist() if 'unna' in col]

    df = df.drop(drop_cols, 1)
    
    rename_col = [col for col in df.columns.tolist() if 'arrecadado_ate' in col]
    
    if rename_col != []:
        df = df.rename(columns = {rename_col[0]:'arrecadado'})
    
        numeric_cols = ['arrecadado']

        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col].str.replace('.','').str.replace(',','.'), errors='coerce')
            
    else:
        numeric_cols = ['previsto_do_ano']

        for col in numeric_cols:
            df[col] = pd.to_numeric(df[col].str.replace('.','').str.replace(',','.'), errors='coerce')
    
    df['ano'] = int(file[-8:-4])
    
    df.to_csv(clean_path+file, index=False, encoding='utf-8')

In [7]:
# import glob, os    
# df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "data/orcamento/clean/receita_arrecadada/*.csv"))))

### Receita Arrecadada

In [73]:
#cria o objeto do schema e da tabela
tb = Table('receita_arrecadada','br_sp_gov_orcamento')


In [74]:
# cria aquivos de config na pasta bases (preencher antes de dar publish), cri bucket no storage e cria tabela no staging
tb.create('data/orcamento/clean/receita_arrecadada//',if_exists="replace")

In [75]:
# publica a tabela no projeto basedosdados com as configs atuais, tabem serve para atualizar as configs
tb.publish('replace')

In [48]:
#update na descrição da tabela
tb.update()

In [ ]:
#update na descricao do dataset
Dataset('br_sp_gov_orcamento').update()

### Receita Prevista

In [8]:
import glob, os    

df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', "data/orcamento/clean/receita_prevista/*.csv"))))


In [9]:
tb = Table('receita_prevista','br_sp_gov_orcamento')


In [10]:
tb.create('data/orcamento/clean/receita_prevista/',if_exists="replace")

Uploading files: 100%|██████████| 10/10 [00:06<00:00,  1.51it/s]


In [17]:
tb.publish('replace')

In [50]:
# tb.update()

In [13]:
df['previsto_do_ano']

0         121000.0
1         273100.0
2           3430.0
3             10.0
4             10.0
           ...    
1744      420000.0
1745    15600000.0
1746          10.0
1747          10.0
1748          10.0
Name: previsto_do_ano, Length: 16297, dtype: float64

In [14]:
df.shape

(16297, 7)

In [15]:
df.head()

,orgao,gestao,unidade_gestora,fonte_de_recursos,receita,previsto_do_ano,ano
0,01000 - ASSEMBLEIA LEGISLATIVA,00001 - GOVERNO DO ESTADO DE SAO PAULO,010030 - FED-ASSEMBLEIA LEGISLATIVA EST.SP.,003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES,13119901 - OUTRAS RECEITAS DE ALUGUEIS DO ESTADO,121000.0,2010
1,01000 - ASSEMBLEIA LEGISLATIVA,00001 - GOVERNO DO ESTADO DE SAO PAULO,010030 - FED-ASSEMBLEIA LEGISLATIVA EST.SP.,003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES,13250101 - FUNDO DE INVESTIMENTO FINANCEIRO - FIF - TES,273100.0,2010
2,01000 - ASSEMBLEIA LEGISLATIVA,00001 - GOVERNO DO ESTADO DE SAO PAULO,010030 - FED-ASSEMBLEIA LEGISLATIVA EST.SP.,003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES,16009901 - OUTROS SERVICOS DO ESTADO,3430.0,2010
3,01000 - ASSEMBLEIA LEGISLATIVA,00001 - GOVERNO DO ESTADO DE SAO PAULO,010030 - FED-ASSEMBLEIA LEGISLATIVA EST.SP.,003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES,17219902 - DEMAIS TRANSFERENCIAS DA UNIAO,10.0,2010
4,01000 - ASSEMBLEIA LEGISLATIVA,00001 - GOVERNO DO ESTADO DE SAO PAULO,010030 - FED-ASSEMBLEIA LEGISLATIVA EST.SP.,003 - RECURSOS VINCULADOS-FUNDO ESPECIAL DE DESPES,17229901 - DOACOES DOS ESTADOS,10.0,2010


## Alesp

#### Despesas

In [206]:
raw_path = 'data/alesp/raw/gastos/'
clean_path = 'data/alesp/clean/gastos/'
files = os.listdir(raw_path)

In [207]:
df = pd.read_csv(raw_path+files[0], dtype={'CNPJ':'str'})
df = df.drop('Data',1)
df.columns = manipulation.normalize_cols(df.columns)
df['cnpj'] = df['cnpj'].str.replace('O','0')

In [58]:
for ano in df['ano'].unique():
    mask = df['ano']==ano
    dd = df[mask]
    
    partitioned_path = clean_path + f'ano={ano}/'
    if os.path.isdir(partitioned_path)==False:
        os.makedirs(partitioned_path)
    else:
        pass
    
    dd = dd.drop('ano',1)
    dd.to_csv(partitioned_path + 'gastos_gabinete.csv',index=False, encoding='utf-8')

In [64]:
tb = Table('despesas_gabinete','br_sp_alesp')

In [80]:
tb.create('data/alesp/clean/gastos/',partitioned=True)

In [81]:
# tb.append('data/alesp/clean/gastos/',partitions=True ,if_exists='replace')

In [79]:
# tb.publish('replace')

In [68]:
# Dataset('br_sp_alesp').update()

#### Servidores

In [209]:
raw_path = 'data/alesp/raw/servidores/'
clean_path = 'data/alesp/clean/servidores/'
files = os.listdir(raw_path)

#### Deputados

In [210]:
file = 'deputados_alesp.csv'
df = pd.read_csv(raw_path+file)

In [211]:
file = 'deputados_alesp.csv'
df = pd.read_csv(raw_path+file)

rename_cols = {
    'idDeputado':'id_deputado',
    'nomeParlamentar':'nome_parlamentar',
    'placaVeiculo':'placa_veiculo',
    'homePage':'home_page',
    'IdSPL':'id_spl',
#     '':'',
}

df = df.rename(columns=rename_cols).drop('numero_deputados',1)

cols = ['matricula', 'nome_parlamentar', 'aniversario', 'partido', 'situacao',
       'email', 'sala', 'placa_veiculo', 'home_page', 'andar', 'id_deputado',
       'id_spl']

df = df[cols]

df['aniversario'] = df['aniversario'].apply(lambda x: x if x!='null/null' else np.nan)

situacao = {
'REL':'Renunciou para assumir outro cargo eletivo',
'OUT':'Outros',
'FAL':'Falecido',
'REN':'Renunciou',
'LIC':'Licenciado',
'EXE':'No exercício do mandato',
'CAS':'Cassado',
'   ':'Não categorizado',
}
df['situacao'] = df['situacao'].map(situacao)

df['andar'] = df['andar'].str.replace('º','').str.replace(' andar','').str.replace('Téreo','Térreo').str.replace('Terreo','Térreo')

df.to_csv(clean_path+'deputados.csv',index=False, encoding='utf-8')

In [221]:
tb = Table('deputados','br_sp_alesp')

In [222]:
# tb.create('data/alesp/clean/servidores/deputados.csv',if_exists='replace')

In [223]:
tb.publish('replace')

#### Assessores Parlamentares e Lideranca

In [212]:
deputados = pd.read_csv(clean_path + 'deputados.csv', dtype={'matricula':str})

file = 'assessores_parlamentares.csv'
df = pd.read_csv(raw_path+file)
df['lotacao_tipo'] = 'GABINETE DO DEPUTADO'

file = 'assessores_lideranca.csv'
dd = pd.read_csv(raw_path+file)
dd['lotacao_tipo'] = 'GABINETE DA LIDERANCA'
dd = dd[df.columns]

df = pd.concat([df,dd],0)

df.columns = manipulation.normalize_cols(df.columns)


In [213]:
cols = ['deputado', 'partido', 'servidor', 'cargo', 'lotacao','lotacao_tipo', 'regime']

df = df[cols].rename(columns={'deputado':'nome_parlamentar'})

df = deputados[['matricula','nome_parlamentar']].merge(df, on='nome_parlamentar', how='outer')

df.to_csv(clean_path+'assessores.csv',index=False, encoding='utf-8')

In [225]:
tb = Table('assessores','br_sp_alesp')

In [224]:
# tb.create('data/alesp/clean/servidores/assessores.csv',if_exists='replace')

In [226]:
tb.publish('replace')

#### tramitacao

In [91]:
raw_path = '../data/alesp/raw/tramitacao/'
clean_path = '../data/alesp/clean/tramitacao/'
files = os.listdir(raw_path)

In [92]:
for file in files:
    df = pd.read_csv(raw_path+file)
    df.columns = manipulation.normalize_cols(df.columns)
    df.to_csv(clean_path+file, index=False, encoding='utf-8')

### Seguranca

#### atividade

In [3]:
raw_path = 'data/seguranca/raw/atividade/'
clean_path = 'data/seguranca/clean/atividade/'
files = os.listdir(raw_path)


ibge_code = pd.read_csv('data/seguranca/raw/ssp_codigo_ibge.csv')
mes_dict = {
    'Jan':1,
    'Fev':2,
    'Mar':3,
    'Abr':4,
    'Mai':5,
    'Jun':6,
    'Jul':7,
    'Ago':8,
    'Set':9,
    'Out':10,
    'Nov':11,
    'Dez':12,
}

In [4]:
for file in files:
    df = pd.read_csv(raw_path+file)
    df = df.merge(ibge_code,on='municipio', how='inner')

    df.columns = manipulation.normalize_cols(df.columns).str.replace('__','_')


    df = df.rename(columns = 
                   {'ocorrencias_de_apreensao_de_entorpecentes1':'ocorrencias_de_apreensao_de_entorpecentes',
                    'geocodigo':'id_municipio',
                    'regiao':'regiao_ssp'
                   }
                  )
    
    df['id_estado'] = 35
    cols = ['id_municipio','id_estado','ano', 'mes','regiao_ssp',
           'ocorrencias_de_porte_de_entorpecentes',
           'ocorrencias_de_trafico_de_entorpecentes',
           'ocorrencias_de_apreensao_de_entorpecentes',
           'ocorrencias_de_porte_ilegal_de_arma',
           'no_de_armas_de_fogo_apreendidas', 'no_de_flagrantes_lavrados',
           'no_de_infratores_apreendidos_em_flagrante',
           'no_de_infratores_apreendidos_por_mandado',
           'no_de_pessoas_presas_em_flagrante', 'no_de_pessoas_presas_por_mandado',
           'no_de_prisoes_efetuadas', 'no_de_veiculos_recuperados',
           'tot_de_inqueritos_policiais_instaurados']

    df = df[cols]
    df['mes'] = df['mes'].map(mes_dict)
    
    df.to_csv(clean_path+file, index=False, encoding='utf-8')

In [2]:
tb = Table('produtividade_policial','br_sp_gov_ssp')

In [3]:
# tb.create('data/seguranca/clean/atividade/',if_exists="replace")

In [4]:
tb.publish('replace')

In [5]:
tb.update()

#### ocorrencias

In [6]:
raw_path = 'data/seguranca/raw/ocorrencias/'
clean_path = 'data/seguranca/clean/ocorrencias/'
files = os.listdir(raw_path)


ibge_code = pd.read_csv('data/seguranca/raw/ssp_codigo_ibge.csv')
mes_dict = {
    'Jan':1,
    'Fev':2,
    'Mar':3,
    'Abr':4,
    'Mai':5,
    'Jun':6,
    'Jul':7,
    'Ago':8,
    'Set':9,
    'Out':10,
    'Nov':11,
    'Dez':12,
}

In [7]:
for file in files:
    df = pd.read_csv(raw_path+file)
    
    df = df.merge(ibge_code,on='municipio', how='inner')

    df.columns = manipulation.normalize_cols(df.columns).str.replace('__','_')

    df = df.rename(columns = 
                   {'geocodigo':'id_municipio',
                    'regiao':'regiao_ssp',
                    'homicidio_doloso_2':'homicidio_doloso',
                    'no_de_vitimas_em_homicidio_doloso_3':'no_de_vitimas_em_homicidio_doloso',
                    'total_de_estupro_4':'total_de_estupro',
                    'total_de_roubo_outros_1':'total_de_roubo_outros',
    #                 '':'',
    #                 '':'',
                   }
                  )
    df['id_estado'] = 35
    df['mes'] = df['mes'].map(mes_dict)

    cols = ['id_municipio', 'id_estado', 'ano', 'mes', 'regiao_ssp',
           'homicidio_doloso', 'no_de_vitimas_em_homicidio_doloso',
           'homicidio_doloso_por_acidente_de_transito',
           'no_de_vitimas_em_homicidio_doloso_por_acidente_de_transito',
           'homicidio_culposo_por_acidente_de_transito',
           'homicidio_culposo_outros', 'tentativa_de_homicidio',
           'lesao_corporal_seguida_de_morte', 'lesao_corporal_dolosa',
           'lesao_corporal_culposa_por_acidente_de_transito',
           'lesao_corporal_culposa_outras', 'latrocinio',
           'no_de_vitimas_em_latrocinio', 'total_de_estupro', 'estupro',
           'estupro_de_vulneravel', 'total_de_roubo_outros', 'roubo_outros',
           'roubo_de_veiculo', 'roubo_a_banco', 'roubo_de_carga', 'furto_outros',
           'furto_de_veiculo']

    df = df[cols]
    
    df.to_csv(clean_path+file, index=False, encoding='utf-8')

In [8]:
tb = Table('ocorrecencias_registradas','br_sp_gov_ssp')

In [9]:
tb.create('data/seguranca/clean/ocorrencias/',if_exists="replace")

Uploading files: 100%|██████████| 19/19 [00:11<00:00,  1.68it/s]


In [10]:
tb.publish('replace')

In [11]:
tb.update()

In [12]:
Dataset('br_sp_gov_ssp').update()

In [14]:
s = Storage('br_sp_ssp_seguranca')

AttributeError: 'NoneType' object has no attribute 'replace'

### Universidades

In [227]:
raw_path = 'data/universidades/raw/'
clean_path = 'data/universidades/clean/'
files = os.listdir(raw_path)

## Downloads

In [29]:
import basedosdados as bd

In [30]:
query = """
                    SELECT *
                    FROM  `basedosdados.br_sp_alesp.despesas_gabinete` LIMIT 10
                    """

In [ ]:
bd.download()

In [229]:
bd.read_sql(query)

Downloading: 100%|██████████| 10/10 [00:04<00:00,  2.46rows/s]


,id_estado,ano,mes,matricula,deputado,cnpj,fornecedor,tipo,valor
0,35,2002,3,300249,NEWTON BRANDÃO,5973844127,POSTO JARDIM DO SHOPPING LTA.,A - COMBUSTÍVEIS E LUBRIFICANTES,10
1,35,2002,3,300278,CELSO TANAUI,61736115,AUTO PODTO 800 LTDA,A - COMBUSTÍVEIS E LUBRIFICANTES,20
2,35,2002,3,300278,CELSO TANAUI,5817877142,AUTO SERVIÇO MONTE SERRAT LTDA,A - COMBUSTÍVEIS E LUBRIFICANTES,20
3,35,2002,3,300278,CELSO TANAUI,5776444187,GAVA & FILHO LTDA,A - COMBUSTÍVEIS E LUBRIFICANTES,77
4,35,2002,3,300278,CELSO TANAUI,56824129,POSTO DE SERVIÇO SANTA TEREZINHA LTDA,A - COMBUSTÍVEIS E LUBRIFICANTES,87
5,35,2002,3,300278,CELSO TANAUI,51552167,AUTO POSTO SANTANA,A - COMBUSTÍVEIS E LUBRIFICANTES,78
6,35,2002,3,300278,CELSO TANAUI,52464961,POSTO DE SERVIÇO PAUMAR LTDA,A - COMBUSTÍVEIS E LUBRIFICANTES,72
7,35,2002,3,300278,CELSO TANAUI,49557335,SUPER POSTO FLOR DE GOIÁS,A - COMBUSTÍVEIS E LUBRIFICANTES,122
8,35,2002,3,300278,CELSO TANAUI,482184,DI-FLORENÇA DERIVADOS PETROLEO LTDA,A - COMBUSTÍVEIS E LUBRIFICANTES,10
9,35,2002,3,300278,CELSO TANAUI,46886186,LANCHES MARISTELA LTDA.,A - COMBUSTÍVEIS E LUBRIFICANTES,30


In [4]:
df = bd.read_table('br_sp_ssp_seguranca','produtividade_policial', limit=10)

Downloading: 100%|██████████| 10/10 [00:03<00:00,  2.94rows/s]


In [5]:
df

,id_municipio,id_estado,ano,mes,regiao_ssp,ocorrencias_de_porte_de_entorpecentes,ocorrencias_de_trafico_de_entorpecentes,ocorrencias_de_apreensao_de_entorpecentes,ocorrencias_de_porte_ilegal_de_arma,no_de_armas_de_fogo_apreendidas,no_de_flagrantes_lavrados,no_de_infratores_apreendidos_em_flagrante,no_de_infratores_apreendidos_por_mandado,no_de_pessoas_presas_em_flagrante,no_de_pessoas_presas_por_mandado,no_de_prisoes_efetuadas,no_de_veiculos_recuperados,tot_de_inqueritos_policiais_instaurados
0,3500105,35,2003,1,Presidente Prudente,3,1,1,1,NaN,NaN,0,0,NaN,2,NaN,NaN,NaN
1,3500105,35,2003,2,Presidente Prudente,2,0,1,1,NaN,NaN,0,0,NaN,4,NaN,NaN,NaN
2,3500105,35,2003,3,Presidente Prudente,4,0,0,0,NaN,NaN,3,0,NaN,0,NaN,NaN,NaN
3,3500105,35,2003,4,Presidente Prudente,4,1,0,2,NaN,NaN,0,0,NaN,3,NaN,NaN,NaN
4,3500105,35,2003,5,Presidente Prudente,4,2,2,1,NaN,NaN,1,0,NaN,1,NaN,NaN,NaN
5,3500105,35,2003,6,Presidente Prudente,2,0,0,0,NaN,NaN,0,0,NaN,9,NaN,NaN,NaN
6,3500105,35,2003,7,Presidente Prudente,0,1,1,1,NaN,NaN,0,0,NaN,1,NaN,NaN,NaN
7,3500105,35,2003,8,Presidente Prudente,0,1,0,0,NaN,NaN,0,0,NaN,0,NaN,NaN,NaN
8,3500105,35,2003,9,Presidente Prudente,2,3,1,0,NaN,NaN,3,0,NaN,1,NaN,NaN,NaN
9,3500105,35,2003,10,Presidente Prudente,3,1,0,0,NaN,NaN,2,0,NaN,1,NaN,NaN,NaN


In [ ]:
bd.download()

In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('/home/m/Downloads/RAIS_VINC_PUB_SP.txt',sep=';',nrows=20kkkk)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe7 in position 9: invalid continuation byte